In [8]:
import pandas as pd
from utils.func import remove_outliers, price_per_square_meter_per_postal_code

In [9]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

combined_df = pd.concat([train, test], axis=0, ignore_index=True)


In [10]:
train = train.convert_dtypes()



In [11]:
from pandas import CategoricalDtype

kit_cat= CategoricalDtype(categories=['not installed', 'usa uninstalled', 'semi equipped', 'usa semi equipped', 'hyper equipped', 'usa hyper equipped', 'installed', 'usa installed'], ordered=True)
train["Kitchen"] = train["Kitchen"].fillna("not installed")
train["Kitchen"] = train["Kitchen"].astype(kit_cat).cat.codes

building_state_type = pd.CategoricalDtype(
    categories=[
        "not known",
        "to be done up",
        "to restore",
        "to renovate",
        "just renovated",
        "good",
        "as new",
    ],
    ordered=True,
)
train["StateOfBuilding"] = train["StateOfBuilding"].fillna("not known")
train["StateOfBuilding"] = (
    train["StateOfBuilding"].astype(building_state_type).cat.codes
)

heating_cat= CategoricalDtype(categories=['not known', 'fueloil', 'gas','carbon', 'wood', 'pellet', 'electric',  'solar'], ordered=True)
train["Heating"] = train["Heating"].fillna("not known")
train["Heating"] = (
    train["Heating"].astype(heating_cat).cat.codes
)
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 35863 entries, 95172 to 57397
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Url                35863 non-null  string 
 1   Openfire           35863 non-null  boolean
 2   Furnished          35863 non-null  boolean
 3   Terrace            35863 non-null  boolean
 4   Garden             35863 non-null  boolean
 5   PropertyId         35863 non-null  Int64  
 6   TypeOfProperty     35863 non-null  Int64  
 7   PostalCode         35863 non-null  Int64  
 8   SubtypeOfProperty  35863 non-null  string 
 9   TypeOfSale         35863 non-null  Int64  
 10  Price              35863 non-null  Int64  
 11  Kitchen            35863 non-null  int8   
 12  StateOfBuilding    35863 non-null  int8   
 13  Heating            35863 non-null  int8   
 14  Bedrooms           25645 non-null  Int64  
 15  SurfaceOfGood      23754 non-null  Int64  
 16  SwimmingPool       3586

In [12]:
train = remove_outliers(train, 'Price', 0.05, 0.95)

train = price_per_square_meter_per_postal_code(train)

train.dropna(subset=['PricePerSquareMeter'],axis=0)

e:\BeCode\AI\immo-eliza-train-test-alexandre\utils\func.py:25: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x["Price"].mean() / (x["SurfaceOfGood"] + x["LivingArea"]).mean()


      PostalCode  PricePerSquareMeter
0              0           357.494407
1            443           813.953488
2           1000           302.100291
3           1020           336.269145
4           1030           927.572559
...          ...                  ...
1111        9988            33.668557
1112        9990           638.429880
1113        9991          1744.027174
1114        9992           360.634697
1115        9999          2706.060606

[1116 rows x 2 columns]
Index(['Url', 'Openfire', 'Furnished', 'Terrace', 'Garden', 'PropertyId',
       'TypeOfProperty', 'PostalCode', 'SubtypeOfProperty', 'TypeOfSale',
       'Price', 'Kitchen', 'StateOfBuilding', 'Heating', 'Bedrooms',
       'SurfaceOfGood', 'SwimmingPool', 'NumberOfFacades', 'LivingArea',
       'ConstructionYear', 'GardenArea', 'PricePerSquareMeter'],
      dtype='object')


,Url,Openfire,Furnished,Terrace,Garden,PropertyId,TypeOfProperty,PostalCode,SubtypeOfProperty,TypeOfSale,...,StateOfBuilding,Heating,Bedrooms,SurfaceOfGood,SwimmingPool,NumberOfFacades,LivingArea,ConstructionYear,GardenArea,PricePerSquareMeter
0,https://www.immoweb.be/en/classified/apartment...,False,True,True,False,10626267,2,7050,apartment,2,...,0,2,2,<NA>,False,2,82,2022,<NA>,79.445787
1,https://www.immoweb.be/en/classified/apartment...,False,False,True,False,8411151,2,1652,apartment,1,...,0,0,1,<NA>,False,<NA>,45,<NA>,<NA>,257.011781
2,https://www.immoweb.be/en/classified/apartment...,False,False,True,False,10781669,2,1030,apartment,2,...,4,1,2,<NA>,False,2,80,<NA>,<NA>,927.572559
3,https://www.immoweb.be/en/classified/house/for...,False,False,True,True,10687773,1,5660,villa,1,...,0,6,3,908,False,4,140,1976,450,103.858453
4,https://www.immoweb.be/en/classified/house/for...,False,False,False,True,10736244,1,8400,villa,1,...,5,0,4,365,False,4,165,<NA>,59,2225.385965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35510,https://www.immoweb.be/en/classified/industry/...,False,False,False,False,10280562,6,1082,industrial premises,2,...,0,0,<NA>,1482,False,2,450,<NA>,<NA>,726.417554
35511,https://www.immoweb.be/en/classified/apartment...,False,False,True,False,10758687,2,4800,apartment,2,...,4,2,2,<NA>,False,2,100,<NA>,<NA>,247.670727
35512,https://www.immoweb.be/en/classified/apartment...,False,False,True,False,10494221,2,6030,apartment,1,...,1,2,2,<NA>,False,2,90,<NA>,<NA>,351.652838
35513,https://www.immoweb.be/en/classified/apartment...,False,False,True,False,10506950,2,9300,duplex unit,1,...,0,0,3,<NA>,False,4,110,2022,<NA>,742.899175


In [13]:
train['SurfaceOfGood'] = train['SurfaceOfGood'].fillna(train['LivingArea'])
train = train.drop(columns=[
    'Url', 
    'SubtypeOfProperty',
    'PropertyId', 
    # 'Kitchen', 
    # 'StateOfBuilding', 
    # 'Heating',
    # 'Price/m2', 
    'GardenArea', 
    'NumberOfFacades', 
    'ConstructionYear',
    'LivingArea'
])

In [14]:
train = train.dropna()
train.to_parquet('train.parquet')